In [1]:
!pip install passlib # hashing passwords
!pip install python-jose # for jwt (tokens)
!pip install schedule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 KB 2.6 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 KB 2.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import os
import io
import json

import requests

import telebot


import schedule
from threading import Thread
from time import sleep


from dotenv import load_dotenv
load_dotenv()

True

In [3]:
JULIA_MEM_API_URL = 'http://127.0.0.1:8000'
# JULIA_MEM_API_URL = 'http://julia-mem-api.fly.dev'

# teleboot
STEPANOVS_FAMILY_BOT_TOKEN = os.environ.get('STEPANOVS_FAMILY_BOT_TOKEN', None)
STEPANOVS_FAMILY_BOT_CHAT_ID = os.environ.get('STEPANOVS_FAMILY_BOT_CHAT_ID', None)
FAMILY_CHAT_ID = os.environ.get('FAMILY_CHAT_ID', None)

In [4]:
# from PIL import Image
# fn = io.BytesIO()
# fn.write(mem_random.content)
# Image.open(fn)

In [6]:
def get_julia_mem(mem_type='random'):
    '''mem_type= 'new', 'random'
        return byte
    '''
    url_mem = f'{JULIA_MEM_API_URL}/mem/{mem_type}'
    
    username = os.getenv('JULIA_MEM_API_USERNAME')
    password = os.getenv('JULIA_MEM_API_PASSWORD')

    url_token = f'{JULIA_MEM_API_URL}/token'
    headers={'accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'username':username, 'password':password}
    response = requests.post(url_token, headers=headers, data=data)
    token_json = json.loads(response.content)
    access_token = token_json["access_token"]
    token_type = token_json["token_type"]

    response_mem = requests.get(url=url_mem, headers={'accept': 'application/json', 'Authorization': f'{token_type} {access_token}'})
    
    if response_mem.status_code == 200:
        return response_mem.content
    return None

In [7]:
def send_julia_mem_to_telegram(bot, telebot_chat_id, byte_mem):
    bot.send_photo(telebot_chat_id, photo=byte_mem)
    return None

In [8]:
def schedule_checker():
    while True:
        schedule.run_pending()
        sleep(1)

In [9]:
def schedule_function_to_run():
    byte_mem = get_julia_mem(mem_type='new')
    if byte_mem:
        send_julia_mem_to_telegram(bot, telebot_chat_id, byte_mem)


In [10]:
telebot_token = STEPANOVS_FAMILY_BOT_TOKEN
telebot_chat_id = STEPANOVS_FAMILY_BOT_CHAT_ID # FAMILY_CHAT_ID
bot = telebot.TeleBot(telebot_token)

In [14]:
# Create the job in schedule.
schedule.every(20).seconds.do(schedule_function_to_run)

# Spin up a thread to run the schedule check so it doesn't block your bot.
# This will take the function schedule_checker which will check every second
# to see if the scheduled job needs to be ran.
Thread(target=schedule_checker).start() 

# And then of course, start your server.
bot.polling()